In [1]:
!pip install opencv-python

In [2]:
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input

from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot as plt
import numpy as np
from numpy.linalg import norm
import cv2
import pickle
from tqdm import tqdm

In [3]:
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import GlobalMaxPooling2D
from keras.models import model_from_json,Sequential

from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from numpy.linalg import norm
import os
from tqdm import tqdm
import pickle
import cv2

In [4]:
feature_list = np.array(pickle.load(open("transfer_learning_recomm._pkl/embeddings_transfer.pkl","rb")))
filenames = pickle.load(open("transfer_learning_recomm._pkl/filenames_transfer.pkl","rb"))

In [5]:
file = open("myModel_arc_transfer.json")
loaded_model = file.read()
file.close()

In [6]:
loaded_model_final = model_from_json(loaded_model)
loaded_model_final.load_weights("myModel_wts_transfer.h5")

In [7]:
model = Sequential()
model.add(loaded_model_final.layers[0])
model.add(loaded_model_final.layers[1])

In [8]:
model.trainable=False

In [9]:
def extract_features(img_path,model):
    # load the image
    img = image.load_img(img_path,target_size=(224,224))
    # img -> array
    img_array = image.img_to_array(img)
    # keras only takes images in batches
    expanded_img_array = np.expand_dims(img_array,axis=0)
    # preprocess img according to ResNet50
    processed_img = preprocess_input(expanded_img_array)
    # extract the features
    result = model.predict(processed_img).flatten()
    # l2 norm.
    normalized_result = result/norm(result)
    
    return normalized_result

In [10]:
## Test data

In [11]:
test = []

for file in os.listdir("test_images"):
    if file not in [".ipynb_checkpoints"]:
        test.append(os.path.join("test_images",file))

In [12]:
test_feature_list = []

In [13]:
for file in tqdm(test):
    test_feature_list.append(extract_features(file,model))

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

1/1 [==============================] - 3s 3s/step


 20%|████████████████▊                                                                   | 1/5 [00:02<00:11,  2.95s/it]

1/1 [==============================] - 0s 243ms/step


 40%|█████████████████████████████████▌                                                  | 2/5 [00:03<00:04,  1.42s/it]

...

 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:03<00:01,  1.09it/s]

1/1 [==============================] - 0s 217ms/step


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:03<00:00,  1.45it/s]

1/1 [==============================] - 0s 303ms/step


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.14it/s]


In [14]:
neighbors = NearestNeighbors(n_neighbors=6,algorithm='brute',metric='euclidean')

In [15]:
neighbors.fit(feature_list)

NearestNeighbors(algorithm='brute', metric='euclidean', n_neighbors=6)

In [24]:
distances,indices = neighbors.kneighbors([test_feature_list[0]])

In [25]:
for file in indices[0][1:6]:
    print(filenames[file])

images\1165.jpg
images\53414.jpg
images\31742.jpg
images\50356.jpg
images\19646.jpg


In [27]:
for file in indices[0][1:6]:
    temp_img = cv2.imread(filenames[file])
    cv2.imshow('output',cv2.resize(temp_img,(128,128)))
    cv2.waitKey(0)